In [ ]:
import json
from langchain_chroma import Chroma
from langchain_text_splitters import (
    RecursiveJsonSplitter,
)
from langchain_openai.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

#전처리된 데이터 10개 약 20만 7천개
#마지막 파일빼고는 24000개씩 데이터 적재
load_dotenv()
for i in range(1, 10):
    print(i)
    with open(f'./data/processed_merged_data{i}.json', 'r',  encoding='utf-8') as j:
        data = json.load(j)
    json_splitter = RecursiveJsonSplitter(max_chunk_size=1300)
    documents = json_splitter.create_documents(texts=data, ensure_ascii=False)
    db = Chroma.from_documents(
        documents=documents,
        embedding=OpenAIEmbeddings(model='text-embedding-3-small', show_progress_bar=True),
        collection_name='books',
        persist_directory='./db'
    )
    